# Set-up

In [1]:
import os
import sys
import scanpy as sc
import muon as mu

import seaborn as sns

sys.path.append("/cellar/users/aklie/opt/gene_program_evaluation/src/inference")
from factor_analysis import run_factor_analysis

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paths
path_data = "/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/iPSC_EC.h5mu"
path_configs = "/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/factor_analysis/config"
path_out = "/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/factor_analysis"

# Load data

In [3]:
# if filepath ends in .h5ad
if path_data.endswith(".h5ad"):
    data = sc.read_h5ad(path_data)
elif path_data.endswith(".h5mu"):
    data = mu.read_h5mu(path_data)
data

/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


MuData object with n_obs × n_vars = 92284 × 5451
  1 modality
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'

# (Optional) Make configs for running outside of the notebook
```yaml
run_factor_analysis_:
    n_components = 5
    random_state = 1234
```

In [44]:
# Configs look like above, make a separate one for 5, 10, 15, 20, 25, 30, 35, 40, 45, 50 100
'''
run_factor_analysis_:
    n_components = 5
    random_state = 1234
'''
def make_factor_analysis_config(path_out, path_configs, n_components):
    path_config = os.path.join(path_configs, f"factor_analysis_{n_components}.gin")
    config = '''run_factor_analysis_:\n\tn_components = {n_components}\n\trandom_state = 1234'''
    config = config.format(n_components=n_components)
    with open(path_config, "w") as f:
        f.write(config)
    print(f"Saved config to {path_config}")
    return path_config

n_components = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
for n in n_components:
    make_factor_analysis_config(path_out, path_configs, n)

Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K5.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K10.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K15.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K20.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K25.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K30.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examples/bin/factor_analysis/config/factor_analysis_K35.gin
Saved config to /cellar/users/aklie/opt/gene_program_evaluation/dashapp/examp

# Run factor analysis

In [4]:
n_components = [100]
data_key = "rna"
layer = "norm10k"

In [5]:
# Run the factor analysis
for n in n_components:
    path_config = os.path.join(path_configs, f"factor_analysis_{n}.gin")
    run_factor_analysis(
        mdata=data,
        prog_key=f"factor_analysis_{n}",
        data_key=data_key,
        layer=layer,
        config_path=path_config,
        inplace=True
    )

INFO:root:Normalising and log transforming count data is reccomended before running FA.


In [6]:
# Check it
data

MuData object with n_obs × n_vars = 92284 × 5451
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    factor_analysis_100:	92284 x 100
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      uns:	'loadings_genes'
      varm:	'loadings'

In [7]:
# Save
data.write(os.path.join(path_out, "factor_analysis_100.h5mu"))

/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


# Quick eval

In [8]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [9]:
fa = mu.read_h5mu(os.path.join(path_out, "factor_analysis_100.h5mu"))
fa

/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


MuData object with n_obs × n_vars = 92284 × 5551
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    factor_analysis_100:	92284 x 100
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      uns:	'loadings_genes'
      varm:	'loadings'

In [10]:
# Grab the factor loadings
adata = fa.mod["factor_analysis_100"].copy()
adata.obsm["X_umap"] = fa.mod["rna"].obsm["X_umap"].copy()
adata

AnnData object with n_obs × n_vars = 92284 × 100
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
    uns: 'loadings_genes'
    obsm: 'X_umap'
    varm: 'loadings'

In [13]:
# Create a divergent colormap from grey to the current color using matplotlib
cmap = plt.cm.get_cmap("Reds")
with plt.rc_context({"figure.figsize": (4, 4)}):
    sc.pl.umap(adata, color=adata.var_names.tolist()[0], cmap=cmap, show=False, vmax="p99.5")

/tmp/ipykernel_2155237/4142146940.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap("Reds")


# DONE!

---